# Zapytania złożone

Przed rozpoczęciem pracy konieczne jest skonfigurowanie bazy oraz załadowanie danych:
```ruby
$:.unshift "."
require 'db_setup'
require 'db_load'
```

In [1]:
$:.unshift "."
require 'db_setup'
require 'db_load'


==  AddIndicesMigration: migrating ============================================
==  AddIndicesMigration: migrated (0.0000s) ===================================



true

W dalszych zadaniach będzie wykonywać polecenia korzystając z następujących klas, zmapowanych na odpowiadające im 
table w bazie danych:
```ruby
class Author < ActiveRecord::Base
  # name      (string)
  # surname   (string)
  # born      (datetime) 
  # died      (datetime)
  # image_url (string)
  
  has_many :books
end

class Book < ActiveRecord::Base
  # title     (string)
  # author    (Author)
  # published (integer)

  belongs_to :author
end
```

ActiveRecord udostępnia interfejs pozwalający wyszukiwać dane w bazie również z wykorzystaniem związków zdefiniowanych pomiędzy
klasami. Do definiowania tych związków służą polecenia `has_many`, `belongs_to`, `has_one` oraz `has_and_belongs_to_many`.
Przykładowo jeśli chcemy dowiedzieć się kto był autorem pierwszej książki wykonujemy zapytanie:
```ruby
book = Book.first
puts book.title
puts book.author
```

In [2]:
book = Book.first
puts book.title
puts book.author

Végjáték (regény)
#<Author:0x00000002b19e08>


Niestety prezentowany wynik nie jest zbyt użyteczny. Ponieważ jednak metoda `author` zwraca obiekt, możemy na nim wywoływać 
metody - tak samo jak w przypadku autora załadowanego bezpośrednio z wykorzystaniem metod zdefiniowanych dla klasy `Author`
```ruby
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"
```

In [3]:
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"

Végjáték (regény)
Orson Scott Card


Co więcej, jeśli mamy do czynienia ze związkiem jeden-do-wiele bądź wiele-do-wiele, możemy wywyływać wszystkie dotychczas poznane 
metody na kolekcji powiązanych obiektów. Istotnie ułatwia to wyszukiwanie potrzebnych danych:
```ruby
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end
```

In [4]:
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end

Anna Karenina
Azazel (powieść)
Anna Karenina


[#<Book id: 33, title: "Anna Karenina", language: nil, author_id: 64, published: 1877, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

In [22]:
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

Anna Karenina
Azazel (powieść)


[#<Book id: 33, title: "Anna Karenina", language: nil, author_id: 64, published: 1877, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Book id: 34, title: "Azazel (powieść)", language: nil, author_id: 65, published: 1998, image_url: nil>]

Różnica pomiędzy tymi zapytaniami polega na tym, że w pierwszym przypadku szukamy książki, których tytuł zaczyna się na "A" 
w całym zbiorze książek, a w drugi przypadku wśród książek napisanych przez Tołstoja.

Aby umożliwić formatowanie wyników zapytania, IRuby udostępnia metodę `IRuby.html`, która pozwala na wyświetlanie kodu
HTML. Przykładowo
```ruby
IRuby.html "<h1>Test HTML-a</h1>"
```

In [6]:
IRuby.html "<h1>Test HTML-a</h1>"

"<h1>Test HTML-a</h1>"

Dzięki temu będziemy mogli formatować wyniki zapytań, a wszczególności wyświetlać obrazy przedstwiające autorów oraz książki. 
Należy jednak wziąć pod uwagę, że nie wszystkie książki oraz nie wszyscy autorzy posiadają odpowiadające im obrazki.
Aby wyświetlić imię, nazwisko oraz obraz pierwszego autora piszemy:
```ruby
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
```

In [30]:
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"

"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>"

Możemy również zdefniować funkcję, np. `display_author`, której zadaniem będzie wyświetlenie autora w tej postaci. 
Jedyny problem polega na tym, że funkcja `IRuby.html` może być wywoływana tylko raz. Dlatego jeśli chcemy wyświetlić kilku autorów,
musimy połączyć odpowiadające im kody HTML i wynik przekazać do funkcji `html`
```ruby
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document
```

In [8]:
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document

"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><h1>Andrzej Sapkowski</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/>"

## Zadania

### Zadanie 1

Wyświetl wszystkich autorów, którzy urodzili się przed rokiem 1900.

In [9]:
authors = Author.where('born < ?',Date.parse("1899-12-31")).all
if authors
  authors.each do |author|
    puts author.name + " " + author.surname 
    #IRuby.html "<h1> #{author.name} #{author.surname}</h1>"

  end
  else
    puts "Nie ma"
  end
#IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"

Juliusz Słowacki
Henryk Sienkiewicz
Eliza Orzeszkowa
Adam Mickiweicz
Jack London
Mikołaj Kopernik
Maria Franciszka Kozłowska
William Szekspir
John Ronald Reuel Tolkien
Aleksander Dumas (ojciec)
Tadeusz Dołęga-Mostowicz
Vladimir Nabokov
Adolf Hitler
Benedykt Chmielowski
Aldous Huxley
Honoré de Balzac
Franz Kafka
Howard Phillips Lovecraft
Lew Tołstoj
Ilja Ilf
Gabriela Zapolska
Clive Staples Lewis
Adam Mickiewicz
Iwan Turgieniew
Lucy Maud Montgomery
Aleksander Majkowski


[#<Author id: 17, name: "Juliusz", surname: "Słowacki", born: "1809-09-04", died: "1849-04-03", image_url: nil>, #<Author id: 18, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-15", image_url: nil>, #<Author id: 19, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-18", image_url: nil>, #<Author id: 39, name: "Adam", surname: "Mickiweicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 42, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 46, name: "Mikołaj", surname: "Kopernik", born: "1473-02-19", died: "1543-05-24", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 47, name: "Maria Franciszka", surname: "Kozłowska", born: "1862-05-27", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 49, name: "William", surname: "Szekspir", born: "

### Zadanie 2

Zmodyfikuj funkcję wyświetlającą autorów tak aby obejmowała ich datę urodzin i śmierci w formacie `dzień.miesiąc.rok`. Następnie wyświetl wszystkich autorów,
którzy zmarli w XX wieku.

In [21]:
authors = Author.where(died: (Date.parse("1901-1-1")..Date.parse("2000-12-31"))).all
if authors
  authors.each do |author|
    puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")} #{author.died.strftime("%d-%m-%Y")} " 


  end
  else
    puts "Nie ma"
  end

Henryk Sienkiewicz 05-05-1846 15-11-1916 
Eliza Orzeszkowa 06-06-1841 18-05-1910 
Jack London 12-01-1876 22-11-1916 
Antoni Gołubiew 25-02-1907 27-06-1979 
George Orwell 25-06-1903 21-01-1950 
John Ronald Reuel Tolkien 03-01-1892 02-09-1973 
Tadeusz Dołęga-Mostowicz 10-08-1898 20-09-1939 
Vladimir Nabokov 22-04-1899 02-07-1977 
Adolf Hitler 20-04-1889 30-04-1945 
Aldous Huxley 26-07-1894 22-11-1963 
Franz Kafka 03-07-1883 03-06-1924 
Howard Phillips Lovecraft 20-08-1890 15-03-1937 
Karol Olgierd Borchardt 25-03-1905 20-05-1986 
Lew Tołstoj 09-09-1828 20-11-1910 
Janusz Zajdel 15-08-1938 19-07-1985 
Isaac Asimov 04-10-1919 06-04-1992 
Ilja Ilf 15-10-1897 13-04-1937 
Jewgienij Pietrow (pisarz) 13-12-1903 02-07-1942 
Gabriela Zapolska 30-03-1857 17-12-1921 
Clive Staples Lewis 29-11-1898 22-11-1963 
Lucy Maud Montgomery 30-11-1874 24-04-1942 
Alfred Szklarski 21-01-1912 09-04-1992 
Aleksander Majkowski 17-07-1876 10-02-1938 
Ian Fleming 28-05-1908 12-08-1964 


[#<Author id: 18, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-15", image_url: nil>, #<Author id: 19, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-18", image_url: nil>, #<Author id: 42, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 45, name: "Antoni", surname: "Gołubiew", born: "1907-02-25", died: "1979-06-27", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 48, name: "George", surname: "Orwell", born: "1903-06-25", died: "1950-01-21", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 51, name: "John Ronald Reuel", surname: "Tolkien", born: "1892-01-03", died: "1973-09-02", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 53, name: "Tadeusz", surname: "Dołęga-Mostowicz", born: "1898-08-10", died: "1939-09-20", image_url: "ht

### Zadanie 3

Wyświetl 5 najmłodszych autorów, którzy posiadają przypisane zdjęcie.

In [41]:
authors = Author.where("image_url is not null").limit(5).order("born desc")
  authors_str = ""
  authors.each do |author|
  #puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")} #{author.died.strftime("%d-%m-%Y")} " 
  authors_str << "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end
IRuby.html authors_str

"<h1>Andrzej Pilipiuk</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Andrzej_Pilipiuk-Polcon2006.jpg?width=300'/><h1>J. K. Rowling</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/J._K._Rowling_2010.jpg?width=300'/><h1>Rafał A. Ziemkiewicz</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Rafał_Ziemkiewicz.jpg?width=300'/><h1>Boris Akunin</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Boris_Akunin_2012.jpg?width=300'/><h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>"

### Zadanie 4

Zdefiniuj funkcję `display_book`, która będzie zwracała fragment kodu HTML obejmujący tytuł książki, rok jej wydania oraz
przypisany obrazek.
Wyświetl najwcześniej wydaną książkę.

In [86]:
def display_book(book)
  "<h1>#{book.title} #{book.published}</h1><img src='#{book.image_url}'/>"
end
book = Book.where("image_url is not null").where("published is not null").order("published").first

IRuby.html display_book(book)


"<h1>Ojcowie i dzieci 0</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Otsy1880.jpg?width=300'/>"

### Zadanie 5

In [60]:
Book.all


#<ActiveRecord::Relation [#<Book id: 1, title: "Végjáték (regény)", language: nil, author_id: 40, published: nil, image_url: nil>, #<Book id: 2, title: "Pani Jeziora (powieść)", language: nil, author_id: 41, published: 1999, image_url: nil>, #<Book id: 3, title: "Bellew Zawierucha", language: nil, author_id: 42, published: 1912, image_url: nil>, #<Book id: 4, title: "Fałszywy trop (powieść)", language: nil, author_id: 43, published: 1995, image_url: nil>, #<Book id: 5, title: "Bajki robotów", language: nil, author_id: 44, published: 1964, image_url: nil>, #<Book id: 6, title: "Bolesław Chrobry (powieść)", language: nil, author_id: 45, published: 1947, image_url: nil>, #<Book id: 7, title: "De revolutionibus orbium coelestium", language: nil, author_id: 46, published: 1543, image_url: nil>, #<Book id: 8, title: "Dzieło Wielkiego Miłosierdzia", language: nil, author_id: 47, published: 1922, image_url: nil>, #<Book id: 9, title: "Folwark zwierzęcy", language: nil, author_id: 48, published

Wyświetl wszystkie książki, które nie mają przypisanego roku wydania.

In [98]:
def display_book1(book)
  "<h1>#{book.title}</h1><img src='#{book.author.image_url}'/>"
end
books = Book.where("published is null").all
puts books.size
str = ""
books.each do |book|
  str << display_book1(book)
end

IRuby.html str

2


"<h1>Végjáték (regény)</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><h1>Hamlet</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/>"

In [2]:
Book.where('published is null').each do |book|
  if book.published == nil
    puts "#{book.title} brak roku wydania"
  else
    puts "#{book.title} #{book.published}"
  end
end

Végjáték (regény) brak roku wydania
Hamlet brak roku wydania


[#<Book id: 1, title: "Végjáték (regény)", language: nil, author_id: 40, published: nil, image_url: nil>, #<Book id: 10, title: "Hamlet", language: nil, author_id: 49, published: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

### Zadanie 6

Zmodyfikuj funkcję `display_author` tak, aby wyświetlała również wszystkie książki napisane przez autora. Użyj znacznika `<ul>`.
Korzystając z tej funkcji wyświetl opis Stanisława Lema.

In [110]:
def display_author(author)
  author = Author.where(surname: author).first
  str = "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
  author.books.each do |book|
    str << "<ul><li><i>\"#{book.title}\"</i></li></ul>"
  end
  str
end

document = display_author("Lem")

IRuby.html document

"<h1>Stanisław Lem</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><ul><li><i>\"Bajki robotów\"</i></li></ul><ul><li><i>\"Solaris (powieść)\"</i></li></ul><ul><li><i>\"Bezsenność (zbiór opowiadań)\"</i></li></ul><ul><li><i>\"Głos Pana\"</i></li></ul><ul><li><i>\"Inwazja z Aldebarana (zbiór opowiadań)\"</i></li></ul><ul><li><i>\"Katar (powieść)\"</i></li></ul><ul><li><i>\"Księga robotów\"</i></li></ul><ul><li><i>\"Maska (zbiór dzieł)\"</i></li></ul><ul><li><i>\"Niezwyciężony i inne opowiadania\"</i></li></ul><ul><li><i>\"Noc księżycowa\"</i></li></ul><ul><li><i>\"Pamiętnik znaleziony w wannie\"</i></li></ul><ul><li><i>\"Polowanie (zbiór opowiadań)\"</i></li></ul><ul><li><i>\"Powtórka (zbiór dzieł)\"</i></li></ul><ul><li><i>\"Pożytek ze smoka\"</i></li></ul><ul><li><i>\"Rasa drapieżców. Teksty ostatnie\"</i></li></ul><ul><li><i>\"Sezam i inne opowiadania\"</i></li></ul><ul><li><i>\"Summa technologiae\"</i></li></ul><ul><li><i>\"Wizja lokalna (powieść)\"</i></li></ul>"

### Zadanie 7

Wśród książek napisanych przez Lema wyświetl wszystkie, które zawieraja w tytule słowo "zbiór".

In [100]:
def display_author(author)
  author = Author.where(surname: author).first
  str = "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
  books = author.books.where("title like '%zbiór%'")
  books.each do |book|
   str << "<ul><li><i>\"#{book.title}\"</i></li></ul>"
  end
  str
end

document = display_author("Lem")

IRuby.html document

"<h1>Stanisław Lem</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><ul><li><i>\"Bezsenność (zbiór opowiadań)\"</i></li></ul><ul><li><i>\"Inwazja z Aldebarana (zbiór opowiadań)\"</i></li></ul><ul><li><i>\"Maska (zbiór dzieł)\"</i></li></ul><ul><li><i>\"Polowanie (zbiór opowiadań)\"</i></li></ul><ul><li><i>\"Powtórka (zbiór dzieł)\"</i></li></ul>"

### Zadanie 8

Wśród książek napisanych przez Lema wyświetl te, które zostały wydane w roku jego śmierci.

In [99]:
def display_author(author)
  author = Author.where(surname: author).first
  str = "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
  books = author.books.where("published == 2006")
  books.each do |book|
   str << "<ul><li><i>\"#{book.title}\"</i></li></ul>"
  end
  str
end

document = display_author("Lem")

IRuby.html document

"<h1>Stanisław Lem</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><ul><li><i>\"Rasa drapieżców. Teksty ostatnie\"</i></li></ul>"

In [116]:
def display_author(author)
  author = Author.where(surname: author).first
  str = "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
  books = Book.where(author: author)
  books.each do |book|
    if book.published == author.died.strftime("%Y")
    str << "<ul><li><i>\"#{book.title}\"</i></li></ul>"
  end
  end
    str
end
document = display_author("Lem")

IRuby.html document



"<h1>Stanisław Lem</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/>"

### Zadanie 9

Wyświetl wszystkie książki znajdujące się w bazie. Opis powinien zawierać tytuł, imię i nazwisko autora, rok wydania oraz 
obraz przedstawiający książkę (o ile istnieje).

In [7]:
def display_book2(book)
  "<h1><i>\"#{book.title}\"</i> #{book.author.surname}</h1> - #{book.published}<img src='#{book.author.image_url}'/>"
end
books = Book.where("image_url is not null").where("published is not null").all
puts books.size
str = ""
books.each do |book|
  str << display_book2(book)
end

IRuby.html str

26


"<h1><i>\"Folwark zwierzęcy\"</i> Orwell</h1> - 1945<img src='http://commons.wikimedia.org/wiki/Special:FilePath/GeoreOrwell.jpg?width=300'/><h1><i>\"Harry Potter\"</i> Rowling</h1> - 1997<img src='http://commons.wikimedia.org/wiki/Special:FilePath/J._K._Rowling_2010.jpg?width=300'/><h1><i>\"Hrabia Monte Christo\"</i> (ojciec)</h1> - 1844<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Dumas_by_Nadar,_1855.jpg?width=300'/><h1><i>\"Krzyżacy (powieść)\"</i> Sienkiewicz</h1> - 1900<img src=''/><h1><i>\"Mein Kampf\"</i> Hitler</h1> - 1925<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Hitler_portrait_crop.jpg?width=300'/><h1><i>\"Nad Niemnem\"</i> Orzeszkowa</h1> - 1888<img src=''/><h1><i>\"Nowe Ateny\"</i> Chmielowski</h1> - 1745<img src='http://commons.wikimedia.org/wiki/Special:FilePath/POL_COA_Nałęcz.svg?width=300'/><h1><i>\"Ogniem i mieczem\"</i> Sienkiewicz</h1> - 1884<img src=''/><h1><i>\"Ojciec Goriot\"</i> Balzac</h1> - 1835<img src='http://commons.wikimedia.org/wiki/Special:FilePath/HBalzac.jpg?width=300'/><h1><i>\"Proces (powieść)\"</i> Kafka</h1> - 1925<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kafka1906.jpg?width=300'/><h1><i>\"Psałterz Dawidów\"</i> Kochanowski</h1> - 1579<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Jan_Kochanowski.png?width=300'/><h1><i>\"Quo vadis\"</i> Sienkiewicz</h1> - 1896<img src=''/><h1><i>\"Romeo i Julia\"</i> Szekspir</h1> - 1597<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/><h1><i>\"Silmarillion\"</i> Tolkien</h1> - 1977<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tolkien_1916.jpg?width=300'/><h1><i>\"Solaris (powieść)\"</i> Lem</h1> - 1961<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><h1><i>\"Władca Pierścieni\"</i> Tolkien</h1> - 1954<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tolkien_1916.jpg?width=300'/><h1><i>\"Anna Karenina\"</i> Tołstoj</h1> - 1877<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Ilya_Efimovich_Repin_(1844-1930)_-_Portrait_of_Leo_Tolstoy_(1887).jpg?width=300'/><h1><i>\"Dives in misericordia\"</i> II</h1> - 1980<img src='http://commons.wikimedia.org/wiki/Special:FilePath/John_Paul_II_Medal_of_Freedom_2004.jpg?width=300'/><h1><i>\"Dwanaście krzeseł\"</i> Ilf</h1> - 1928<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Ilya_Ilf.jpg?width=300'/><h1><i>\"Księgi narodu polskiego i pielgrzymstwa polskiego\"</i> Mickiewicz</h1> - 1832<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Adam_Mickiewicz_by_Jan_Mieczkowski.jpg?width=300'/><h1><i>\"Ojcowie i dzieci\"</i> Turgieniew</h1> - 0<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Turgenev_by_Repin.jpg?width=300'/><h1><i>\"Sen nocy letniej\"</i> Szekspir</h1> - 1600<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/><h1><i>\"Slavorum apostoli\"</i> II</h1> - 1985<img src='http://commons.wikimedia.org/wiki/Special:FilePath/John_Paul_II_Medal_of_Freedom_2004.jpg?width=300'/><h1><i>\"Ut unum sint\"</i> II</h1> - 1995<img src='http://commons.wikimedia.org/wiki/Special:FilePath/John_Paul_II_Medal_of_Freedom_2004.jpg?width=300'/><h1><i>\"Veritatis splendor\"</i> II</h1> - 1993<img src='http://commons.wikimedia.org/wiki/Special:FilePath/John_Paul_II_Medal_of_Freedom_2004.jpg?width=300'/><h1><i>\"Wojna i pokój\"</i> Tołstoj</h1> - 1865<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Ilya_Efimovich_Repin_(1844-1930)_-_Portrait_of_Leo_Tolstoy_(1887).jpg?width=300'/>"

### Zadanie 10

Wyświetl wszystkich autorów znajdujących się w bazie wraz z opisem zawierającym ich zdjęcie, datę urodzin i śmierci oraz spis 
opublikowanych książek. Pamiętaj o odpowiednim sformatowaniu dat oraz o tym, że nie wszyscy autorzy mają przypisane wszystkie
informacje.

In [131]:
def display_authors
  author = Author.all
  str = ""
  author.each do |author|
    str << "<h1>#{author.name} #{author.surname}</h1> #{author.born} ---#{author.died}<img src='#{author.image_url}'/>"

  author.books.each do |book|
    str << "<ul><li><i>\"#{book.title}\"</i></li></ul>"
  end
end
  str
end

document = display_authors

IRuby.html document

"<h1>Juliusz Słowacki</h1> 1809-09-04 ---1849-04-03<img src=''/><h1>Henryk Sienkiewicz</h1> 1846-05-05 ---1916-11-15<img src=''/><ul><li><i>\"Krzyżacy (powieść)\"</i></li></ul><ul><li><i>\"Ogniem i mieczem\"</i></li></ul><ul><li><i>\"Quo vadis\"</i></li></ul><h1>Eliza Orzeszkowa</h1> 1841-06-06 ---1910-05-18<img src=''/><ul><li><i>\"Nad Niemnem\"</i></li></ul><h1>Adam Mickiweicz</h1> 1798-12-24 ---1855-11-26<img src=''/><h1>Orson Scott Card</h1> 1951-08-24 ---<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><ul><li><i>\"Végjáték (regény)\"</i></li></ul><h1>Andrzej Sapkowski</h1> 1948-06-21 ---<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/><ul><li><i>\"Pani Jeziora (powieść)\"</i></li></ul><h1>Jack London</h1> 1876-01-12 ---1916-11-22<img src='http://commons.wikimedia.org/wiki/Special:FilePath/JackLondon02.jpeg?width=300'/><ul><li><i>\"Bellew Zawierucha\"</i></li></ul><h1>Henning Mankell</h1> 1948-02-03 ---<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Henning_Mankell_3_2011_Shankbone.jpg?width=300'/><ul><li><i>\"Fałszywy trop (powieść)\"</i></li></ul><h1>Stanisław Lem</h1> 1921-09-12 ---2006-03-27<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><ul><li><i>\"Bajki robotów\"</i></li></ul><ul><li><i>\"Solaris (powieść)\"</i></li></ul><ul><li><i>\"Bezsenność (zbiór opowiadań)\"</i></li></ul><ul><li><i>\"Głos Pana\"</i></li></ul><ul><li><i>\"Inwazja z Aldebarana (zbiór opowiadań)\"</i></li></ul><ul><li><i>\"Katar (powieść)\"</i></li></ul><ul><li><i>\"Księga robotów\"</i></li></ul><ul><li><i>\"Maska (zbiór dzieł)\"</i></li></ul><ul><li><i>\"Niezwyciężony i inne opowiadania\"</i></li></ul><ul><li><i>\"Noc księżycowa\"</i></li></ul><ul><li><i>\"Pamiętnik znaleziony w wannie\"</i></li></ul><ul><li><i>\"Polowanie (zbiór opowiadań)\"</i></li></ul><ul><li><i>\"Powtórka (zbiór dzieł)\"</i></li></ul><ul><li><i>\"Pożytek ze smoka\"</i></li></ul><ul><li><i>\"Rasa drapieżców. Teksty ostatnie\"</i></li></ul><ul><li><i>\"Sezam i inne opowiadania\"</i></li></ul><ul><li><i>\"Summa technologiae\"</i></li></ul><ul><li><i>\"Wizja lokalna (powieść)\"</i></li></ul><h1>Antoni Gołubiew</h1> 1907-02-25 ---1979-06-27<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Antoni_Golubiew.jpg?width=300'/><ul><li><i>\"Bolesław Chrobry (powieść)\"</i></li></ul><h1>Mikołaj Kopernik</h1> 1473-02-19 ---1543-05-24<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nikolaus_Kopernikus.jpg?width=300'/><ul><li><i>\"De revolutionibus orbium coelestium\"</i></li></ul><h1>Maria Franciszka Kozłowska</h1> 1862-05-27 ---<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Mateczk_Kozłowska.JPG?width=300'/><ul><li><i>\"Dzieło Wielkiego Miłosierdzia\"</i></li></ul><h1>George Orwell</h1> 1903-06-25 ---1950-01-21<img src='http://commons.wikimedia.org/wiki/Special:FilePath/GeoreOrwell.jpg?width=300'/><ul><li><i>\"Folwark zwierzęcy\"</i></li></ul><ul><li><i>\"Droga na molo w Wigan\"</i></li></ul><h1>William Szekspir</h1> 1564-04-23 ---1616-05-03<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/><ul><li><i>\"Hamlet\"</i></li></ul><ul><li><i>\"Romeo i Julia\"</i></li></ul><ul><li><i>\"Sen nocy letniej\"</i></li></ul><h1>J. K. Rowling</h1> 1965-07-31 ---<img src='http://commons.wikimedia.org/wiki/Special:FilePath/J._K._Rowling_2010.jpg?width=300'/><ul><li><i>\"Harry Potter\"</i></li></ul><h1>John Ronald Reuel Tolkien</h1> 1892-01-03 ---1973-09-02<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tolkien_1916.jpg?width=300'/><ul><li><i>\"Hobbit, czyli tam i z powrotem\"</i></li></ul><ul><li><i>\"Silmarillion\"</i></li></ul><ul><li><i>\"Władca Pierścieni\"</i></li></ul><h1>Aleksander Dumas (ojciec)</h1> 1802-07-24 ---1870-12-05<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Dumas_by_Nadar,_1855.jpg?width=300'/><ul><li><i>\"Hrabi